# Table of Contents
 <p><div class="lev1 toc-item"><a href="#use-RMG-to-get-plog-of-data" data-toc-modified-id="use-RMG-to-get-plog-of-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>use RMG to get plog of data</a></div><div class="lev1 toc-item"><a href="#compare-atkinson-to-quantum" data-toc-modified-id="compare-atkinson-to-quantum-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>compare atkinson to quantum</a></div>

finding branching ratio between nitrate and NO2 + alkoxy under atm conditions. Numbers come from atkinson 1997

In [1]:
import math
import numpy as np
import pandas as pd

In [2]:
def get_nitrate_yield(carbon_number, pressure, temperature, carbon_type,
                     other_functional_groups=None):
    """
    Returns the expected fraction of ROO + NO reaction that forms RONO2
    (instead of RO + NO2).
    
    carbon number is the number of carbon atoms in the compound
    pressure is given in pascals
    temperature in K
    carbon type is a string of 'primary', 'secondary', and 'tertiary'
    other_functional_groups is a list of other functional groups that are known to
    adjust the rate. Currently accepted functional groups are:
        hydroxy group (halves the yield)
    
    
    from Carter and Atkinson 1987
    """
    # derived inputs
    concentration = pressure / 8.314 / temperature /10**6 *6.022e23 #molecule/cm3
    # diven constants
    y_inf = 0.826
    alpha = 1.94e-22 #cm3/mol-1
    beta = 0.97
    m0 = 0.0
    m_inf = 8.1
    F = 0.411
    if carbon_number < 2:
        raise AttributeError('get_nitrate_ratio is only valid for 2 or more carbons. '\
                             'See Atkinson 1997 for more information')
    # calculations
    y0 = alpha * math.exp(beta * carbon_number)
    numerator = y0*concentration * (temperature/300.)**(-m0)
    denominator = y_inf * (temperature/300.)**-m_inf
    zeta = 1 / (1+ (np.log10(numerator / denominator ))**2)
    # final answer
    rate_ratio = (numerator / (1+ numerator/denominator)) * F**zeta
    rate_yield = 1 / (1 / rate_ratio + 1) # go from ka/kb to ka/(ka+kb)
    if carbon_type == 'primary':
        rate_yield *= 0.4
    elif carbon_type == 'tertiary':
        rate_yield *= 0.3
    elif carbon_type != 'secondary':
        raise ValueError('Only primary, secondary and tertiary accepted for carbon_type')
    if other_functional_groups is not None:
        for fg in other_functional_groups:
            if (fg == 'hydroxy') or (fg == 'OH'):
                rate_yield /= 2 #from atkinson 1997 for hydroxy groups formed from alkenes
    return rate_yield

In [3]:
# test get nitrate ratio
# using data of carter and atkinson 1989
data = [(3, 'secondary', 299, 2.37e19, 0.047),
        (4, 'secondary', 299, 2.37e19, 0.083),
        (5, 'secondary', 284, 2.52e19, 0.158),
        (6, 'secondary', 299, 2.37e19, 0.193),
        (6, 'secondary', 281, 1.20e19, 0.179),
        (3, 'primary',   299, 2.37e19, 0.019),
        (5, 'primary',   282, 2.51e19, 0.065),
        (6, 'tertiary',  298, 2.38e19, 0.059)]

for carbon_number, carbon_type, temperature, density, expected_output in data:
    pressure = density * 8.314 * temperature * 10**6 / 6.022e23
    nitrate_yield = get_nitrate_yield(carbon_number, pressure, temperature, carbon_type)
    if abs(nitrate_yield - expected_output) > 0.001:
        print('Model result different than Carter and Atkinson for data: '+str((carbon_number, carbon_type, temperature, density, expected_output)))
        print('Expected {}, got {}'.format(expected_output, nitrate_yield))

# use RMG to get plog of data

In [4]:
from rmgpy.kinetics.arrhenius import Arrhenius, PDepArrhenius
from rmgpy.chemkin import write_kinetics_entry
from rmgpy.reaction import Reaction
from rmgpy.species import Species

In [5]:
number_carbons = 4
carbon_type = 'primary'
peroxy_name = 'PC4H9O2'
nitrate_name = 'PC4H9ONO2'
alkoxy_name = 'PC4H9O'
peroxy_smiles = 'CCCCO[O]'
nitrate_smiles = 'CCCCO[N+](=O)[O-]'
alkoxy_smiles = 'CCCC[O]'
use_total_rate = True
other_functional_groups = ['OH']

In [6]:
# get the alkoxy rates

# original rate (possibly erroniously from atkinson 1997 for h-abstraction of ho2 by peroxy radicals)
#alkoxy_kinetics = Arrhenius(A=(1.1144e11,'cm^3/(mol*s)'),n=0,Ea=(-2583,('cal/mol')))

if use_total_rate:
    #atkinson 1997 for C2+
    total_kinetics = Arrhenius(A=(2.7e-12*6.022e23,'cm^3/(mol*s)'),n=0,Ea=(-360*8.314,('J/mol')))
else:
    #anderlohr 2009
    alkoxy_kinetics = Arrhenius(A=(4.7e12,'cm^3/(mol*s)'),n=0,Ea=(-358,('cal/mol')))


In [7]:
temperatures = np.linspace(250,1250,20)
pressures = np.logspace(3,7,5)

In [8]:
data = np.ndarray((len(temperatures),len(pressures)))
data_alkoxy = np.ndarray((len(temperatures),len(pressures)))
for i1, t in enumerate(temperatures):
    for i2, p in enumerate(pressures):
        nitrate_yield = get_nitrate_yield(number_carbons,p,t,carbon_type,other_functional_groups=other_functional_groups)
        if use_total_rate:
            total_rate = total_kinetics.get_rate_coefficient(t)
            data[i1,i2] = total_rate * nitrate_yield
            data_alkoxy[i1,i2] = total_rate * (1-nitrate_yield)
        else:
            nitrate_to_NO2_ratio = 1 / (1 / nitrate_yield - 1)
            data[i1,i2] = alkoxy_kinetics.get_rate_coefficient(t) * nitrate_to_NO2_ratio

In [9]:
nitrate_rate = PDepArrhenius().fit_to_data(temperatures,pressures, data, 'm^3/(mol*s)')
if use_total_rate:
    alkoxy_rate = PDepArrhenius().fit_to_data(temperatures,pressures, data_alkoxy, 'm^3/(mol*s)')

In [10]:
no = Species().from_smiles('[N]=O')
peroxy = Species(label = peroxy_name).from_smiles(peroxy_smiles)
nitrate = Species(label = nitrate_name).from_smiles(nitrate_smiles)

rxn = Reaction(reactants = [no,peroxy], products = [nitrate], kinetics = nitrate_rate)
print(write_kinetics_entry(rxn, [no,peroxy,nitrate]))
# this is in units of kcal/mol and mol and may need to have the activation energy modified

NO+PC4H9O2=PC4H9ONO2                                1.000e+00 0.000     0.000    
    PLOG/ 0.009869  1.704e+32 -8.087    3.870    /
    PLOG/ 0.098692  2.851e+37 -9.757    4.136    /
    PLOG/ 0.986923  5.489e+38 -10.226   3.355    /
    PLOG/ 9.869233  1.544e+38 -10.104   2.460    /
    PLOG/ 98.692327 1.031e+37 -9.762    1.765    /



In [11]:
if use_total_rate:
    alkoxy = Species(label = alkoxy_name).from_smiles(alkoxy_smiles)
    no2 = Species().from_smiles('N(=O)[O]')
    rxn2 = Reaction(reactants = [no,peroxy], products = [alkoxy, no2], kinetics = alkoxy_rate)
    print(write_kinetics_entry(rxn2, [no,peroxy,alkoxy,no2]))
    # this is in units of kcal/mol and mol and may need to have the activation energy modified

NO+PC4H9O2=PC4H9O+NO2                               1.000e+00 0.000     0.000    
    PLOG/ 0.009869  1.628e+12 -0.000    -0.715   /
    PLOG/ 0.098692  1.671e+12 -0.003    -0.709   /
    PLOG/ 0.986923  2.037e+12 -0.029    -0.670   /
    PLOG/ 9.869233  3.143e+12 -0.086    -0.586   /
    PLOG/ 98.692327 5.280e+12 -0.154    -0.489   /



In [12]:
#nitrate
print(repr(rxn.kinetics))

PDepArrhenius(pressures=([0.01,0.1,1,10,100],'bar'), arrhenius=[Arrhenius(A=(1.70409e+26,'m^3/(mol*s)'), n=-8.08735, Ea=(16.1936,'kJ/mol'), T0=(1,'K'), Tmin=(250,'K'), Tmax=(1250,'K'), comment="""Fitted to 20 data points; dA = *|/ 4.84581, dn = +|- 0.213524, dEa = +|- 0.953612 kJ/mol"""), Arrhenius(A=(2.85071e+31,'m^3/(mol*s)'), n=-9.75679, Ea=(17.3031,'kJ/mol'), T0=(1,'K'), Tmin=(250,'K'), Tmax=(1250,'K'), comment="""Fitted to 20 data points; dA = *|/ 3.27964, dn = +|- 0.160704, dEa = +|- 0.717715 kJ/mol"""), Arrhenius(A=(5.48873e+32,'m^3/(mol*s)'), n=-10.2261, Ea=(14.0354,'kJ/mol'), T0=(1,'K'), Tmin=(250,'K'), Tmax=(1250,'K'), comment="""Fitted to 20 data points; dA = *|/ 2.26894, dn = +|- 0.110856, dEa = +|- 0.49509 kJ/mol"""), Arrhenius(A=(1.54367e+32,'m^3/(mol*s)'), n=-10.1042, Ea=(10.2928,'kJ/mol'), T0=(1,'K'), Tmin=(250,'K'), Tmax=(1250,'K'), comment="""Fitted to 20 data points; dA = *|/ 4.84154, dn = +|- 0.213405, dEa = +|- 0.953079 kJ/mol"""), Arrhenius(A=(1.03115e+31,'m^3/(mo

In [16]:
print(repr(rxn2.kinetics))

PDepArrhenius(pressures=([0.01,0.1,1,10,100],'bar'), arrhenius=[Arrhenius(A=(1.62791e+06,'m^3/(mol*s)'), n=-0.000141688, Ea=(-2.99104,'kJ/mol'), T0=(1,'K'), Tmin=(250,'K'), Tmax=(1250,'K'), comment="""Fitted to 20 data points; dA = *|/ 1.00026, dn = +|- 3.56474e-05, dEa = +|- 0.000159204 kJ/mol"""), Arrhenius(A=(1.67071e+06,'m^3/(mol*s)'), n=-0.00346841, Ea=(-2.96676,'kJ/mol'), T0=(1,'K'), Tmin=(250,'K'), Tmax=(1250,'K'), comment="""Fitted to 20 data points; dA = *|/ 1.00166, dn = +|- 0.000225047, dEa = +|- 0.00100507 kJ/mol"""), Arrhenius(A=(2.03697e+06,'m^3/(mol*s)'), n=-0.0292964, Ea=(-2.80448,'kJ/mol'), T0=(1,'K'), Tmin=(250,'K'), Tmax=(1250,'K'), comment="""Fitted to 20 data points; dA = *|/ 1.02194, dn = +|- 0.00293612, dEa = +|- 0.0131129 kJ/mol"""), Arrhenius(A=(3.14328e+06,'m^3/(mol*s)'), n=-0.0858879, Ea=(-2.45349,'kJ/mol'), T0=(1,'K'), Tmin=(250,'K'), Tmax=(1250,'K'), comment="""Fitted to 20 data points; dA = *|/ 1.06894, dn = +|- 0.00901984, dEa = +|- 0.0402832 kJ/mol"""), 

In [136]:
# ensure the rates add up
errors = np.ndarray((len(temperatures),len(pressures)))
for i1, temperature in enumerate(temperatures):
    for i2, pressure in enumerate(pressures):
        if use_total_rate:
            r1 = rxn.get_rate_coefficient(temperature, pressure)
            r2 = rxn2.get_rate_coefficient(temperature, pressure)
            total = total_kinetics.get_rate_coefficient(temperature)
            error = abs(r1 + r2 - total) / total
            errors[i1, i2] = error

In [137]:
np.median(errors)

0.000763531895444425

In [138]:
np.mean(errors)

0.002924592061216611

In [139]:
np.max(errors)

0.04939315144115557

In [140]:
t_test = 300

In [ ]:
np.max(data)

In [ ]:
np.argmax(data)

In [ ]:
data

In [ ]:
ratios = np.ndarray((len(temperatures),len(pressures)))

for i1, t in enumerate(temperatures):
    for i2, p in enumerate(pressures):
        ratios[i1,i2] = get_nitrate_ratio(number_carbons,p,t)

In [ ]:
pd.DataFrame(index=temperatures, columns = pressures, data=ratios)

# compare atkinson to quantum
Here I output atkinson rates at 300, 800, and 1500 and compare it to the RRCM results by http://dx.doi.org/10.1016/j.comptc.2017.04.015

In [ ]:
import numpy as np

In [ ]:
atkinson_rxn = lambda t: 2.7e-12*np.exp(360./t)
temps = [300,800,1500]
atkinson_rates = np.array([atkinson_rxn(t) for t in temps])

In [ ]:

#these were translated by engague from figure 7 in the paper at low pressure limits.
ng_rates = np.array([2.08e-11,2.47e-11,6.58e-11])

In [ ]:
atkinson_rates

In [ ]:
rate_diff = ng_rates/ atkinson_rates

In [ ]:
rate_diff

It seems like a different rate coefficient is predicted by the quantum and atkinson, with quantum overestimating the rate, which is typical. The rate overestimate is 2 times at atm temperature, 6 times at 800K and 20x at 1500K. 

In [64]:
6e-14*6.022e23

36132000000.0

In [66]:
550 *1.987

1092.8500000000001